In [ ]:
import requests
import json
import math
import pandas as pd
from collections import OrderedDict 

In [ ]:
response = requests.get('https://disease.sh/v3/covid-19/historical?lastdays=all')
pop = pd.read_csv("./population.csv",skiprows=4, index_col="Country Name")
countries = ['Germany','Afgahnistan','Belgium','Italy','UK','USA']

In [ ]:
string = "country,date,cumulative,new,relative\n"
jsonObj = response.json()
for i,country in enumerate(jsonObj):
    if country['country'] in pop.index and country['country'] in countries and not country['province']: #if we have population info for the country
        population =  pop.loc[country['country']]['2019']
        if not math.isnan(population):
            cases = country['timeline']['cases'] #turn cumulative cases from dataset into list for later iteration
            arr = []
            for date in cases.keys():
                obj = {'date': date, 'cumulative': cases[date]}
                arr.append(obj)

            for j,e in enumerate(arr): #compute new cases 
                if j > 0:
                    yesterdayCases = arr[j-1]['cumulative']
                else:
                    yesterdayCases = 0
                e['new'] = e['cumulative'] - yesterdayCases
                    
                sevenDaySum = 0
                daysBack = 7
                while daysBack > 0:
                    if(j >= daysBack):
                        sevenDaySum = sevenDaySum +  arr[j-daysBack]['new']
                    daysBack = daysBack-1
    
                e['relative'] = math.floor(sevenDaySum / population * 100000)
        
                if country['province']:
                    province = "-"+country['province']
                else:
                    province = ""
                if True:
                    string = string + country['country']+ province + ","
                    string = string + e['date'] + ","
                    string = string + str(e['cumulative']) + ","
                    string = string + str(e['new']) + ","
                    string = string + str(e['relative'])
                    string = string + "\n"

            country['cases'] = arr
            del country['timeline']
        else:
            del country
    else:
        del country
        
print(string)

jsonObj

In [ ]:
cases = [0,1,2,3,4,5,6,7,8,9]


for i, case in enumerate(cases):
    sum = 0
    
    j=3
    while j > 0:
        if(i>j):
            print(cases[i-j])
            sum = sum + cases[i-j]
        j = j-1
    print("-->"+str(sum))

In [ ]:
with open('../src/assets/data.json', 'w', encoding='utf-8') as f:    
    json.dump(jsonObj, f, ensure_ascii=False, indent=4)

In [ ]:
#ARCHIVE

jsonObj = response.json()
for i,item in enumerate(jsonObj):
    if item['country'] in pop.index:
        incidence = {}
        countryPop = pop.loc[item['country']]['2019']
        item['population'] = countryPopt
        for j, key in enumerate(item['timeline']['cases'].keys()):
            newCases = item['timeline']['cases']
            if i<1 and j < 10:
                print(j)
            
            absoluteCases = item['timeline']['cases'][key]
            relativeCases = item['timeline']['cases'][key]
            incidence[j] = {'date': key, 'incidence': relativeCases}
       
        item['incidence'] = incidence
        del item['timeline']
    else:
        del item
    